## Hello linear model
This tutorial is part of the Tensorflow getting started documentation at: https://www.tensorflow.org/tutorials/wide

## Convert data into a feature space
Extract a features of a sliding window from a given sample data.

In [140]:
#TODO: Change rotor label to the appropriate flight controls
import csv
from itertools import islice

FEATURE_DATA_FILEPATH = './data/'

feature_csv_columns = ['average']
imu_data_columns = ['gyro_roll', 'gyro_pitch', 'gyro_yaw', 'acc_x', 'acc_y', 'acc_z']

# Transform samples from sliding window to a feature
def feature_average(data_sample):
    fSum = 0;
    nIndex = 0;
    for data in data_sample:
        fSum = fSum + data
        nIndex = nIndex + 1
    return float(fSum / nIndex)

feature_calculations = [feature_average]

# # Iterate through all the features and use each function to extract the features of a sliding window
# def extract_features_from_imu_data_window(features_filepath, data_windows):
#     running_feature_of_sliding_window = defaultdict(list)
#     running_line=""
#     running_lines=[]
#     with open(features_filepath, 'a') as file:
#         running_index = 0;
        
#         for data_window in data_windows:
#             running_lines.append("")
#             print('extracting for ', data_window)
#             for feature in feature_calculations:
#                 running_line += str(feature(data_window))
# #                 if(running_index < len(feature_calculations)):
# #                     running_line += ','
#                 running_lines[running_index] += (running_line + ",")
#             running_line = ""
#             running_index = running_index + 1
        
#         for line in running_lines:
#             print('WRITE LINE', line)
        
# #         print('WRITE LINE', running_line)
#         file.write(running_line)
#         running_line = ""
       

# # Iterate through a sliding window and make a function call to extract features
# def extract_features_from_imu_data_sample(features_filepath, raw_imu_samples, window_length = 6, window_step_forward = 3):
#     # Extract windows from raw IMU sample data
#     print('SAMPLES: ', imu_data_columns)
#     sliding_windows = []
#     for imu_data_column in imu_data_columns:
#         print('EXTRACTING SAMPLES FOR', raw_imu_samples[imu_data_column])
#         data_sample_iterator = iter(raw_imu_samples[imu_data_column])
#         sliding_window = tuple(islice(data_sample_iterator, window_length))
#         print('GOT SLIDING WINDOW ', sliding_window)
#         index = 0

#         for window in data_sample_iterator:
#             if (index % window_step_forward == 0):
#                 #extract_features_from_imu_data_window(features_filepath, sliding_window)
#                 sliding_windows.append(sliding_window)
#                 sliding_window = sliding_window[1:] + (window,)
#             else:
#                 sliding_window = sliding_window[1:] + (window,)
#             index = index + 1
#         if(index % window_step_forward == 0):
#             #extract_features_from_imu_data_window(features_filepath, sliding_window)
#             sliding_windows.append(sliding_window)
#     extract_features_from_imu_data_window(features_filepath, sliding_windows)
    

# # Read a *.csv file and extract the sliding window
# from collections import defaultdict
# def extract_features_from_imu_data_samples_for_label(data_sample_filepath, features_filepath, data_label):
#     with open(data_sample_filepath, 'r') as csv_file:
#         # extract data records by column
#         reader = csv.DictReader(csv_file)
#         running_data = defaultdict(list)
#         #imu_data_columns = ['gyro_roll', 'gyro_pitch', 'gyro_yaw', 'acc_x', 'acc_y', 'acc_z']
#         for row in reader:
#             for imu_data_column in imu_data_columns:
#                 raw_imu_data_sample = row[imu_data_column];
#                 running_data[imu_data_column].append(int(row[imu_data_column]))
#         extract_features_from_imu_data_sample(features_filepath, running_data, 4, 1)












    

# Read a *.csv file and extract the sliding window
import collections
def extract_features_from_imu_data_samples_for_label(data_sample_filepath, features_filepath, data_label):
    with open(data_sample_filepath, 'r') as csv_file:
        # extract data records by row
        reader = csv.DictReader(csv_file)
        sliding_windows = []
        sliding_index = 0
        window_step_forward = 1
        window_length = 4
        
        # extract sliding windows from rows
        sliding_window = []
        for row in reader:
            sliding_window.append(row)
            if(len(sliding_window) == window_length + 1):
                del sliding_window[0]
            if(sliding_index % window_step_forward == 0 and len(sliding_window) == window_length):
                sliding_windows.append(list(sliding_window))
            sliding_index = sliding_index + 1
        
        print('SLIDING WINDOWS', sliding_windows)
        print('SLIDING WINDOWS[0]', sliding_windows[0])
        print('SLIDING WINDOWS[0][0]', sliding_windows[0][0])
        
        running_line = ""
        
        
        window_sequence_accs = []
        for sliding_window in sliding_windows:
            window_sequence_acc = []
            for samples in sliding_window:
                window_sequence_acc.append(samples['acc_x'])
            window_sequence_accs.append(window_sequence_acc)
                
        print('asdf', window_sequence_accs)
#                 for feature in feature_csv_columns:
#                          print('HEY ', sampled_window, 'for', feature)
#                         running_line += str(feature(samples[imu_data_column]))
#                         print(running_line)
#                         running_lines[running_index] += (running_line + ",")
#                     running_line = ""
#                     running_index = running_index + 1
            
                
            
#             if(sliding_index % window_step_forward == 0):
                
#                 sliding_windows.append(sliding_window)
#                 print('GOT SLIDING WINDOW ', sliding_window)
#             else:
#                 sliding_window.append(row)
                
                
        
#         sliding_window_iterator = iter(reader)
#         sliding_window = tuple(islice(sliding_window_iterator, window_length))
#         print('GOT SLIDING WINDOW ', sliding_window)
        
#         index = 0
#         for window in sliding_window_iterator:
#             if (index % window_step_forward == 0):
#                 #extract_features_from_imu_data_window(features_filepath, sliding_window)
#                 sliding_windows.append(sliding_window)
#                 sliding_window = sliding_window[1:] + (window,)
#             else:
#                 sliding_window = sliding_window[1:] + (window,)
#             index = index + 1
#         if(index % window_step_forward == 0):
#             #extract_features_from_imu_data_window(features_filepath, sliding_window)
#             sliding_windows.append(sliding_window)
            
            
            
    # Extract windows from raw IMU sample data
#     print('SAMPLES: ', imu_data_columns)
#     sliding_windows = []
#     for imu_data_column in imu_data_columns:
#         print('EXTRACTING SAMPLES FOR', raw_imu_samples[imu_data_column])
#         data_sample_iterator = iter(raw_imu_samples[imu_data_column])
#         sliding_window = tuple(islice(data_sample_iterator, window_length))
        
#         index = 0
#         for window in data_sample_iterator:
#             if (index % window_step_forward == 0):
#                 #extract_features_from_imu_data_window(features_filepath, sliding_window)
#                 sliding_windows.append(sliding_window)
#                 sliding_window = sliding_window[1:] + (window,)
#             else:
#                 sliding_window = sliding_window[1:] + (window,)
#             index = index + 1
#         if(index % window_step_forward == 0):
#             #extract_features_from_imu_data_window(features_filepath, sliding_window)
#             sliding_windows.append(sliding_window)





# Iterate through the raw IMU data directories and get their labels
import glob
import os
labels = set()
print('Extracting labels for samples in raw data directory:')
for raw_data_dir in glob.glob('./data/**/raw/*', recursive=True):
    labels.add(os.path.basename(raw_data_dir))
print(labels)

#os.remove('./data/features.csv')
with open('./data/features.csv', 'w') as file:
    running_line = ""
    running_index = 1
    for feature_columns in feature_csv_columns:
        for imu_data_column in imu_data_columns:
            running_line += feature_columns + '_' + imu_data_column + ','
            if(running_index < len(feature_csv_columns)):
                running_line += ','
            running_index = running_index + 1
    running_line += '\n'
    file.write(running_line)


# Iterate through each raw IMU data sample and extract their features:
for labelled_features in labels:
    print('Extracting features for:', labelled_features)
    for raw_data_dir in glob.glob('./data/**/raw/' + labelled_features + '/*.csv', recursive=False):
        extract_features_from_imu_data_samples_for_label(raw_data_dir, './data/features.csv', labelled_features)

Extracting labels for samples in raw data directory:
{'rotor3', 'rotor1', 'rotor4', 'rotor2'}
Extracting features for: rotor3
SLIDING WINDOWS [[{'acc_y': '1378', 'acc_x': '315', 'gyro_yaw': '-1179', 'acc_z': '-15682', 'gyro_pitch': '181', 'gyro_roll': '-3661'}, {'acc_y': '1492', 'acc_x': '-35', 'gyro_yaw': '-639', 'acc_z': '-2183', 'gyro_pitch': '-3279', 'gyro_roll': '4684'}, {'acc_y': '-935', 'acc_x': '15', 'gyro_yaw': '-597', 'acc_z': '-9016', 'gyro_pitch': '-2489', 'gyro_roll': '3159'}, {'acc_y': '493', 'acc_x': '-999', 'gyro_yaw': '-1176', 'acc_z': '-2968', 'gyro_pitch': '-2908', 'gyro_roll': '287'}], [{'acc_y': '1492', 'acc_x': '-35', 'gyro_yaw': '-639', 'acc_z': '-2183', 'gyro_pitch': '-3279', 'gyro_roll': '4684'}, {'acc_y': '-935', 'acc_x': '15', 'gyro_yaw': '-597', 'acc_z': '-9016', 'gyro_pitch': '-2489', 'gyro_roll': '3159'}, {'acc_y': '493', 'acc_x': '-999', 'gyro_yaw': '-1176', 'acc_z': '-2968', 'gyro_pitch': '-2908', 'gyro_roll': '287'}, {'acc_y': '-633', 'acc_x': '139', 'g


## Implement a linear classifier

In [ ]:
import tensorflow as tf
import os
import shutil
import sys

model_dir='./tmp/model'
train_data='./sample-data/rotor-tap-sample-data.csv'

# delete the model directory
shutil.rmtree(model_dir, ignore_errors=True)

# declare feature columns within csv
median_gyro_roll = tf.feature_column.numeric_column(key='median-gyro-roll', dtype=tf.float64);
median_gyro_pitch = tf.feature_column.numeric_column(key='median-gyro-pitch', dtype=tf.float64);
median_gyro_yaw = tf.feature_column.numeric_column(key='median-gyro-yaw', dtype=tf.float64);

median_acc_x = tf.feature_column.numeric_column(key='median-acc-x', dtype=tf.float64);
median_acc_y = tf.feature_column.numeric_column(key='median-acc-y', dtype=tf.float64);
median_acc_z = tf.feature_column.numeric_column(key='median-acc-z', dtype=tf.float64);

mean_gyro_roll = tf.feature_column.numeric_column(key='mean-gyro-roll', dtype=tf.float64);
mean_gyro_pitch = tf.feature_column.numeric_column(key='mean-gyro-pitch', dtype=tf.float64);
mean_gyro_yaw = tf.feature_column.numeric_column(key='mean-gyro-yaw', dtype=tf.float64);

# stack feature columns into a single array
imu_window_feature_columns = [median_gyro_roll, median_gyro_pitch, median_gyro_yaw,
        median_acc_x, median_acc_y, median_acc_z,
        mean_gyro_roll, mean_gyro_pitch, mean_gyro_yaw]

run_config=tf.estimator.RunConfig().replace(
    session_config=tf.ConfigProto(device_count={'GPU': 0})
)

def input_fn(data_file):
    assert tf.gfile.Exists(data_file),('%s not found')
    records_default = [['0'], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0],
                       [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
    csv_columns = [
                    'rotor',
                    'median-gyro-roll','median-gyro-pitch','median-gyro-yaw',
                    'median-acc-x','median-acc-y','median-acc-z',
                    'mean-gyro-roll','mean-gyro-pitch','mean-gyro-yaw',
                    'mean-acc-x','mean-acc-y','mean-acc-z',
                    'skew-gyro-roll','skew-gyro-pitch','skew-gyro-yaw',
                    'skew-acc-x','skew-acc-y','skew-acc-z'
                  ]
    
    def parse_csv(value):
        print('PARSING:', data_file)
        columns = tf.decode_csv(value, records_default)
        features = dict(zip(csv_columns, columns))
        labels = features.pop('rotor')
        print('LABELS:', labels)
        return features, labels
    
    dataset = tf.data.TextLineDataset(data_file)
    dataset = dataset.shuffle(200)
    dataset = dataset.map(parse_csv, 4)
    dataset = dataset.batch(200)
    return dataset

# model = tf.estimator.LinearClassifier(
#     model_dir=model_dir,
#     feature_columns=columns,
#     config=run_config,
#     n_classes=4,
#     label_vocabulary=['1', '2', '3', '4']
# )
model = tf.estimator.DNNClassifier(
    model_dir=model_dir,
    feature_columns=imu_window_feature_columns,
    config=run_config,
    hidden_units=[100, 75, 50, 25],
    n_classes=4,
    label_vocabulary=['1', '2', '3', '4']
)



# Train and evaluate the model every `FLAGS.epochs_per_eval` epochs.
for n in range(40 // 2):
    model.train(input_fn=lambda: input_fn(
        train_data))

    results = model.evaluate(input_fn=lambda: input_fn(
        train_data))

    # Display evaluation metrics
    print('Results at epoch', (n + 1) * 2)
    print('-' * 60)





# model.train(input_fn=lambda: input_fn(
#     train_data))

# results = model.evaluate(input_fn=lambda:input_fn(
#     train_data
# ))